In [1]:
import http.client
import ssl
import pandas as pd
import json

# Ignorar a verificação SSL
context = ssl._create_unverified_context()

# Estabelecer conexão com a API
conn = http.client.HTTPSConnection("api-comexstat.mdic.gov.br", context=context)

# Definir o payload para a solicitação POST
payload = {
    "flow": "import",
    "monthDetail": True, 
    "period": {
        "from": "2000-01",
        "to": "2024-12"
    },
    "filters": [
        {
            "filter": "heading",
            "values": [
                "0302", # Peixes frescos ou refrigerados
                "0303", # Peixes congelados
                "0304", # Filés de peixe
                "0305", # Peixes secos, salgados ou em salmoura
                "0306", # Crustáceos
                "0307", # Moluscos
                "0308", # Invertebrados aquáticos
                "0309", # Farinhas, pós e pellets
                "1604", # Crustáceos, moluscos e outros invertebrados aquáticos
                "1605"  # Preparações e conservas de Peixe
            ]
        },
        { 
            "filter": "country", 
            "values": [
                "756",  # Africa do Sul
                "063",  # Argentina
                "069",  # Australia
                "105",  # Brasil
                "149",  # Canadá
                "158",  # Chile
                "160",  # China
                "169",  # Colombia
                "687",  # El Salvador
                "239",  # Equador
                "245",  # Espanha
                "249",  # Estados Unidos
                "267",  # Filipinas
                "275",  # França
                "305",  # Groenlandia
                "365",  # Indonesia
                "379",  # Islandia
                "386",  # Italia
                "399",  # Japao
                "442",  # Lituania
                "474",  # Marrocos
                "493",  # México
                "499",  # Micronesia
                "538",  # Noruega
                "548",  # Nova Zelandia
                "565",  # Omã
                "580",  # Panamá
                "589",  # Peru
                "607",  # Portugal
                "623",  # Quenia
                "628",  # Reino Unido
                "888",  # Republica Dominicana
                "791",  # Singapura
                "750",  # Sri Lanka
                "776",  # Tailandia
                "161",  # Taiwan
                "845",  # Uruguai
                "858"   # Vietna
            ]
        }
                
    ], 
    "details": [
        "country",
        "state",
        "ncm",
        "heading"
    ],
    "metrics": [
        "metricFOB",
        "metricKG"
    ]
}

headers = {
    'Content-Type': "application/json",
    'Accept': "application/json"
}

# Enviar solicitação POST para a API
conn.request("POST", "/general", json.dumps(payload), headers)

# Obter a resposta da API
res = conn.getresponse()
data = res.read().decode("utf-8")

# Converter a resposta em um dicionário Python
response_json = json.loads(data)
data_list = response_json['data']


# Verificar se a chave 'data' existe no dicionário e acessá-la
if 'data' in response_json:
    # Extrair a lista de dicionários contida na chave 'data'
    data_list = response_json['data']
    
    # Usar pd.json_normalize para converter em DataFrame
    df = pd.DataFrame(data_list)
    
    # Verificar se o DataFrame não está vazio
    if not df.empty:
        # Salvar o DataFrame em um arquivo CSV
        caminho_arquivo = 'dados_brutos.csv'
        df.to_csv(caminho_arquivo, index=False)
        print(f"DataFrame salvo com sucesso em '{caminho_arquivo}'.")
    else:
        print("Nenhum dado encontrado.")
else:
    print("A chave 'data' não foi encontrada na resposta JSON.")


DataFrame salvo com sucesso em 'dados_brutos.csv'.


In [2]:
import pandas as pd
import ast

arquivo_comex = 'dados_brutos.csv'
data = pd.read_csv(arquivo_comex)

# Convertendo a string do dicionário para um dicionário Python real
data['list'] = data['list'].apply(ast.literal_eval)

# Normalizando a coluna com dicionários
df = pd.json_normalize(data['list'])

# Concatenando os DataFrames
result = pd.concat([data, df], axis=1)
result.drop('list', axis=1, inplace=True)

# Criando um dicionário de mapeamento dos nomes antigos para os novos nomes
rename_columns = {
    'coNcm': 'Codigo NCM',
    'year': 'Ano',
    'monthNumber': 'Mês',
    'heading': 'Posição (SH4)',
    'country': 'País',
    'state': 'Estado',
    'ncm': 'Descrição NCM',
    'metricFOB': 'Valor FOB',
    'metricKG': 'Peso em KG'
}

# Renomeando as colunas
result.rename(columns=rename_columns, inplace=True)

# Ordenando os dados pelo número do mês
result = result.sort_values(by=['Posição (SH4)', 'Ano', 'Mês'])


# Salvando o DataFrame em um novo arquivo CSV
result.to_csv('dados_tabulados.csv', index=False)
print(f"DataFrame salvo com sucesso em {caminho_arquivo}")    




DataFrame salvo com sucesso em dados_brutos.csv
